In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2024-02-10 18:47:49.846115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-10 18:47:50.409230: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 18:47:52.307197: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/aaron/anaconda3/envs/tf/lib/
2024-02-10 18:47:52.307733: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.

In [2]:
train_data_dir = 'dataset'

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [4]:
train_generator = train_datagen.flow_from_directory(train_data_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 3001 images belonging to 2 classes.


In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping

# Load InceptionV3 model
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add your custom fully connected layers
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create the new model
model = tf.keras.models.Model(base_model.input, x)

# Add EarlyStopping callback to monitor training loss
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

# Compile the model with the callback
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['acc'])

# Train the model with early stopping using generator
inception_hist = model.fit(train_generator, 
                           steps_per_epoch=20, 
                           epochs=10, 
                           callbacks=[early_stopping])

# Print the final accuracy
print("Final Accuracy: ", inception_hist.history['acc'][-1])


2024-02-10 18:47:54.417793: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-10 18:47:54.458578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-10 18:47:54.458908: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-10 18:47:54.459647: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/10


2024-02-10 18:48:05.795886: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2024-02-10 18:48:08.647359: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-10 18:48:08.649641: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-10 18:48:08.650395: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-02-10 18:48:08.651132: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-10 18:48:08.651579: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-02-10 18:48:09.773434: I tensorflow/stream_executor/cuda/c

20/20 [==============================] - 13s 178ms/step - loss: 0.5548 - acc: 0.7875
Epoch 2/10
20/20 [==============================] - 3s 164ms/step - loss: 0.5538 - acc: 0.7800
Epoch 3/10
20/20 [==============================] - 3s 164ms/step - loss: 0.5744 - acc: 0.7675
Epoch 4/10
20/20 [==============================] - 3s 157ms/step - loss: 0.5574 - acc: 0.7650
Epoch 5/10
20/20 [==============================] - 3s 164ms/step - loss: 0.4854 - acc: 0.8025
Epoch 6/10
20/20 [==============================] - 3s 163ms/step - loss: 0.4393 - acc: 0.8300
Epoch 7/10
20/20 [==============================] - 3s 162ms/step - loss: 0.5170 - acc: 0.7612
Epoch 8/10
20/20 [==============================] - 3s 154ms/step - loss: 0.5224 - acc: 0.7825
Epoch 9/10
20/20 [==============================] - 3s 157ms/step - loss: 0.4665 - acc: 0.8110
Final Accuracy:  0.8110235929489136
